In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
#District Summary Calculations

# Calculate the total number of schools
total_schools = school_data_complete["school_name"].nunique()

# Calculate the total number of students
total_students = len(school_data_complete["Student ID"])

# Calculate the total budget  
schools = school_data_complete.groupby(["school_name"])
school_budget = schools['budget'].mean()
school_budget_df = pd.DataFrame({"School Budget": school_budget})
tot_school_budget = school_budget_df["School Budget"].sum()

# Calculate the average math score
tot_math_avg = round(school_data_complete["math_score"].sum() \
                     / total_students,3)

# Calculate the average reading score
tot_read_avg = round(school_data_complete["reading_score"].sum() \
                     / total_students,3)

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_df = school_data_complete.loc[school_data_complete["math_score"] >= 70]
tot_passing_math = len(passing_math_df)
percent_passing_math = tot_passing_math / total_students *100
school_data_complete["Passing Math"] = np.where(school_data_complete["math_score"] >= 70,1,0)
total_pass_math_2 = school_data_complete["Passing Math"].sum()

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_read_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
tot_passing_read = len(passing_read_df)
percent_passing_read = tot_passing_read / total_students *100
school_data_complete["Passing Read"] = np.where(school_data_complete["reading_score"] >= 70,1,0)


# Calculate the percentage of students who passed math and reading (% Overall Passing)
passing_both_df = school_data_complete.loc[((school_data_complete["math_score"] >= 70) & \
                                            (school_data_complete["reading_score"] >= 70))]
tot_passing_both = len(passing_both_df)
percent_passing_both = len(passing_both_df) / total_students *100
school_data_complete["Passing Overall"] = np.where(((school_data_complete["math_score"] >= 70) & \
                                                    (school_data_complete["reading_score"] >= 70)),1,0) 

# Create a dataframe to hold the above results
summary_df = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students],  \
                           "Total Budget": [tot_school_budget], "Average Math Score": [tot_math_avg],  \
                           "Average Reading Score": [tot_read_avg], "% Passing Math":percent_passing_math,  \
                           "% Passing Reading":percent_passing_read,"% Overall Passing":percent_passing_both,})


# formatting
summary_df['Total Students'] = summary_df['Total Students'].map("{:,}".format)
summary_df['Total Budget']=summary_df['Total Budget'].astype(float).map("${:,.2f}".format)
summary_df["% Passing Math"] = summary_df["% Passing Math"].map("{:.3f}%".format) 
summary_df["% Passing Reading"] = summary_df["% Passing Reading"].map("{:.3f}%".format) 
summary_df["% Overall Passing"] = summary_df["% Overall Passing"].map("{:.3f}%".format) 


## District Summary

In [3]:
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985,81.878,74.981%,85.805%,65.172%


In [4]:
#group by & calcs for by by school table
schools_type = school_data_complete.groupby(["school_name","type"])
total_students_per = schools_type["Student ID"].count()
total_budget_per = schools_type["budget"].mean()
per_student_budget = total_budget_per / total_students_per
avg_math_score = schools_type["math_score"].mean()
avg_read_score = schools_type["reading_score"].mean()

#Create new table with schools
schools_type_df = pd.DataFrame({"Total Students": total_students_per,"Total School Budget": total_budget_per,  \
                                "Per Student Budget":per_student_budget,"Average Math Score": avg_math_score, \
                                "Average Reading Score": avg_read_score})

schools_type_df = schools_type_df.reset_index()
schools_type_df = schools_type_df.set_index(["school_name"])
schools_type_df = schools_type_df.rename(columns={"type":"School Type"})

#formatting
schools_type_df['Total Students'] =schools_type_df['Total Students'].map("{:,}".format)
schools_type_df['Total School Budget']=schools_type_df['Total School Budget'].astype(float).map("${:,.2f}".format)
schools_type_df['Per Student Budget']=schools_type_df['Per Student Budget'].astype(float).map("${:,.2f}".format)

per_student_budget

school_name            type    
Bailey High School     District    628.0
Cabrera High School    Charter     582.0
Figueroa High School   District    639.0
Ford High School       District    644.0
Griffin High School    Charter     625.0
Hernandez High School  District    652.0
Holden High School     Charter     581.0
Huang High School      District    655.0
Johnson High School    District    650.0
Pena High School       Charter     609.0
Rodriguez High School  District    637.0
Shelton High School    Charter     600.0
Thomas High School     Charter     638.0
Wilson High School     Charter     578.0
Wright High School     Charter     583.0
dtype: float64

In [5]:
# Calculate the percentage of students with a passing math score (70 or greater) per school
passing_math_per_school = passing_math_df.groupby(["school_name"])
number_passing_math = passing_math_per_school["Student ID"].count()

passing_math_per_df = pd.DataFrame({"Number Passing Math":number_passing_math})

# Calculate the percentage of students with a passing reading score (70 or greater) per school
passing_read_per_df = passing_read_df.groupby(["school_name"])
number_passing_read = passing_read_per_df["Student ID"].count()

passing_read_per_df = pd.DataFrame({"Number Passing Reading":number_passing_read})

# Calculate the percentage of students who passed math and reading (% Overall Passing)
passing_both_per_school = passing_both_df.groupby(["school_name"])
number_passing_both = passing_both_per_school["Student ID"].count()

passing_both_per_df = pd.DataFrame({"Number Passing Overall":number_passing_both})

In [6]:
#merge passing tables with by school table
schools_type_math_df = pd.merge(schools_type_df,passing_math_per_df,on="school_name")
schools_type_math_read_df = pd.merge(schools_type_math_df,passing_read_per_df,on="school_name")
schools_type_both_df = pd.merge(schools_type_math_read_df,passing_both_per_df,on="school_name")
schools_type_both_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Number Passing Math,Number Passing Reading,Number Passing Overall
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,3318,4077,2719
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,1749,1803,1697
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,1946,2381,1569
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,1871,2172,1487
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,1371,1426,1330
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,3094,3748,2481
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,395,411,381
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,1916,2372,1561
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,3145,3867,2549


In [7]:
#Calc % Passing Math & add to to table
percent_passing_math = schools_type_both_df["Number Passing Math"].values \
                                                  / total_students_per *100

schools_type_both_df["% Passing Math"] = np.array(percent_passing_math)

#Calc % Passing Read & add to to table
percent_passing_read = schools_type_both_df["Number Passing Reading"].values \
                                                  / total_students_per *100

schools_type_both_df["% Passing Reading"] = np.array(percent_passing_read)

#Calc % Passing Overall & add to to table
percent_passing_overall = schools_type_both_df["Number Passing Overall"].values \
                                                  / total_students_per *100

schools_type_both_df["% Overall Passing"] = np.array(percent_passing_overall)

#formatting
schools_type_both_df["% Passing Math"] = schools_type_both_df["% Passing Math"].map("{:.3f}%".format) 
schools_type_both_df["% Passing Reading"] = schools_type_both_df["% Passing Reading"].map("{:.3f}%".format)
schools_type_both_df["% Overall Passing"] = schools_type_both_df["% Overall Passing"].map("{:.3f}%".format) 

#drop calc only columns
schools_sum_df = schools_type_both_df.drop(columns=['Number Passing Math','Number Passing Reading','Number Passing Overall'])

## School Summary

In [8]:
schools_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680%,81.933%,54.642%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133%,97.040%,91.335%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988%,80.739%,53.204%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.310%,79.299%,54.290%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392%,97.139%,90.599%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.753%,80.863%,53.528%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.506%,96.253%,89.227%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.684%,81.316%,53.514%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.058%,81.222%,53.539%


## Top Performing Schools (By % Overall Passing)

In [9]:
#Sort and display the top five performing schools by % overall passing
schools_top5_df = schools_sum_df.sort_values("% Overall Passing",ascending = False).head(5)

schools_top5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133%,97.040%,91.335%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272%,97.309%,90.948%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392%,97.139%,90.599%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.868%,96.540%,90.583%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.595%,95.946%,90.541%


## Bottom Performing Schools (By % Overall Passing)

In [10]:
#Sort and display the five worst-performing schools by % overall passing
schools_top5_df = schools_sum_df.sort_values("% Overall Passing",ascending = True).head(5)

schools_top5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.367%,80.220%,52.988%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988%,80.739%,53.204%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.684%,81.316%,53.514%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.753%,80.863%,53.528%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.058%,81.222%,53.539%


In [25]:
#find average math & reading scores for 9th graders only
def selector_9m(row):
    if row["grade"] == "9th":
        return row["math_score"]

def selector_9r(row):
    if row["grade"] == "9th":
        return row["reading_score"]

school_data_complete["9th Math Scores"] = school_data_complete.apply(lambda row: selector_9m(row), axis=1)
school_data_complete["9th Reading Scores"] = school_data_complete.apply(lambda row: selector_9r(row), axis=1)

#pick only relavent rows & columns
school_data_9th = school_data_complete.dropna(subset=["9th Math Scores"])
school_data_9th = school_data_9th[["Student ID","school_name","grade","9th Math Scores","9th Reading Scores"]]

#group by school & calc average score for 9th grade only
schools_type_9th = school_data_9th.groupby(["school_name"])
total_students_per_9th = schools_type_9th["Student ID"].count()
avg_math_score = schools_type_9th["9th Math Scores"].mean()
avg_read_score = schools_type_9th["9th Reading Scores"].mean()

#Create new table 9th grade only by school
schools_type_9th_df = pd.DataFrame({"9th Avg Math Score": avg_math_score, "9th Avg Reading Score": avg_read_score})

schools_type_9th_df = schools_type_9th_df.reset_index()
schools_type_9th_df

,school_name,9th Avg Math Score,9th Avg Reading Score
0,Bailey High School,77.083676,81.303155
1,Cabrera High School,83.094697,83.676136
2,Figueroa High School,76.403037,81.198598
3,Ford High School,77.361345,80.632653
4,Griffin High School,82.044010,83.369193
5,Hernandez High School,77.438495,80.866860
6,Holden High School,83.787402,83.677165
7,Huang High School,77.027251,81.290284
8,Johnson High School,77.187857,81.260714
9,Pena High School,83.625455,83.807273


In [12]:
#find average math & reading scores for 10th graders only
def selector_10m(row):
    if row["grade"] == "10th":
        return row["math_score"]
    
def selector_10r(row):
    if row["grade"] == "10th":
        return row["reading_score"]

school_data_complete["10th Math Scores"] = school_data_complete.apply(lambda row: selector_10m(row), axis=1)
school_data_complete["10th Reading Scores"] = school_data_complete.apply(lambda row: selector_10r(row), axis=1)


#pick only relavent rows & columns
school_data_10th = school_data_complete.dropna(subset=["10th Math Scores"])
school_data_10th = school_data_10th[["Student ID","school_name","grade","10th Math Scores","10th Reading Scores"]]

#group by school & calc average score for 10th grade only
schools_type_10th = school_data_10th.groupby(["school_name"])
total_students_per_10th = schools_type_10th["Student ID"].count()
avg_math_score = schools_type_10th["10th Math Scores"].mean()
avg_read_score = schools_type_10th["10th Reading Scores"].mean()

#Create new table 9th grade only by school
schools_type_10th_df = pd.DataFrame({"10th Avg Math Score": avg_math_score, "10th Avg Reading Score": avg_read_score})

schools_type_10th_df = schools_type_10th_df.reset_index()
schools_type_10th_df

,school_name,10th Avg Math Score,10th Avg Reading Score
0,Bailey High School,76.996772,80.907183
1,Cabrera High School,83.154506,84.253219
2,Figueroa High School,76.539974,81.408912
3,Ford High School,77.672316,81.262712
4,Griffin High School,84.229064,83.706897
5,Hernandez High School,77.337408,80.660147
6,Holden High School,83.429825,83.324561
7,Huang High School,75.908735,81.512386
8,Johnson High School,76.691117,80.773431
9,Pena High School,83.372000,83.612000


In [27]:
#find average math & reading scores for 11th graders only
def selector_11m(row):
    if row["grade"] == "11th":
        return row["math_score"]

def selector_11r(row):
    if row["grade"] == "11th":
        return row["reading_score"]

school_data_complete["11th Math Scores"] = school_data_complete.apply(lambda row: selector_11m(row), axis=1)
school_data_complete["11th Reading Scores"] = school_data_complete.apply(lambda row: selector_11r(row), axis=1)


#pick only relavent rows & columns
school_data_11th = school_data_complete.dropna(subset=["11th Math Scores"])
school_data_11th = school_data_11th[["Student ID","school_name","grade","11th Math Scores","11th Reading Scores"]]

#group by school & calc average score for 10th grade only
schools_type_11th = school_data_11th.groupby(["school_name"])
total_students_per_11th = schools_type_11th["Student ID"].count()
avg_math_score = schools_type_11th["11th Math Scores"].mean()
avg_read_score = schools_type_11th["11th Reading Scores"].mean()

#Create new table 9th grade only by school
schools_type_11th_df = pd.DataFrame({"11th Avg Math Score": avg_math_score, "11th Avg Reading Score": avg_read_score})

schools_type_11th_df = schools_type_11th_df.reset_index()
schools_type_11th_df

,school_name,11th Avg Math Score,11th Avg Reading Score
0,Bailey High School,77.515588,80.945643
1,Cabrera High School,82.765560,83.788382
2,Figueroa High School,76.884344,80.640339
3,Ford High School,76.918058,80.403642
4,Griffin High School,83.842105,84.288089
5,Hernandez High School,77.136029,81.396140
6,Holden High School,85.000000,83.815534
7,Huang High School,76.446602,81.417476
8,Johnson High School,77.491653,80.616027
9,Pena High School,84.328125,84.335938


In [28]:
#find average math & reading scores for 12th graders only
def selector_12m(row):
    if row["grade"] == "12th":
        return row["math_score"]
    
def selector_12r(row):
    if row["grade"] == "12th":
        return row["reading_score"]

school_data_complete["12th Math Scores"] = school_data_complete.apply(lambda row: selector_12m(row), axis=1)
school_data_complete["12th Reading Scores"] = school_data_complete.apply(lambda row: selector_12r(row), axis=1)


#pick only relavent rows & columns
school_data_12th = school_data_complete.dropna(subset=["12th Math Scores"])
school_data_12th = school_data_12th[["Student ID","school_name","grade","12th Math Scores","12th Reading Scores"]]

#group by school & calc average score for 10th grade only
schools_type_12th = school_data_12th.groupby(["school_name"])
total_students_per_12th = schools_type_12th["Student ID"].count()
avg_math_score = schools_type_12th["12th Math Scores"].mean()
avg_read_score = schools_type_12th["12th Reading Scores"].mean()

#Create new table 9th grade only by school
schools_type_12th_df = pd.DataFrame({"12th Avg Math Score": avg_math_score, "12th Avg Reading Score": avg_read_score})

schools_type_12th_df = schools_type_12th_df.reset_index()
schools_type_12th_df

,school_name,12th Avg Math Score,12th Avg Reading Score
0,Bailey High School,76.492218,80.912451
1,Cabrera High School,83.277487,84.287958
2,Figueroa High School,77.151369,81.384863
3,Ford High School,76.179963,80.662338
4,Griffin High School,83.356164,84.013699
5,Hernandez High School,77.186567,80.857143
6,Holden High School,82.855422,84.698795
7,Huang High School,77.225641,80.305983
8,Johnson High School,76.863248,81.227564
9,Pena High School,84.121547,84.591160


In [29]:
#merge grade tables with each other
schools_type_by_grade_df = pd.merge(schools_type_9th_df,schools_type_10th_df,on="school_name")
schools_type_by_grade_df = pd.merge(schools_type_by_grade_df,schools_type_11th_df,on="school_name")
schools_type_by_grade_df = pd.merge(schools_type_by_grade_df,schools_type_12th_df,on="school_name")
schools_type_by_grade_df = schools_type_by_grade_df.set_index(["school_name"])

#Build Math Scores by Grade table
schools_avg_math_by_grade_df = schools_type_by_grade_df[["9th Avg Math Score","10th Avg Math Score", \
                                                         "11th Avg Math Score","12th Avg Math Score"]]

schools_avg_math_by_grade_df = schools_avg_math_by_grade_df.rename(columns={"9th Avg Math Score":"9th","10th Avg Math Score":"10th", \
                                                                    "11th Avg Math Score":"11th","12th Avg Math Score":"12th"})


#Build Reading Scores by Grade table
schools_avg_read_by_grade_df = schools_type_by_grade_df[["9th Avg Reading Score","10th Avg Reading Score", \
                                                         "11th Avg Reading Score","12th Avg Reading Score"]]

schools_avg_read_by_grade_df = schools_avg_read_by_grade_df.rename(columns={"9th Avg Reading Score":"9th","10th Avg Reading Score":"10th", \
                                                                    "11th Avg Reading Score":"11th","12th Avg Reading Score":"12th"})


## Math Scores by Grade

In [30]:
schools_avg_math_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [31]:
schools_avg_read_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [32]:
schools_by_spend_range_df = pd.merge(school_data_complete,schools_type_df,on="school_name")
schools_by_spend_range_df['Per Student Budget'] = schools_by_spend_range_df['Per Student Budget'] \
                                                  .str.replace('$','').astype(float).astype(int)
#list(schools_by_spend_range_df.columns)

In [33]:
#Perform the same operations as above, based on school size
bins = [0,585,630,645,680]
spending_ranges = ["<585","$586-630","$631-645","$646+"]

schools_by_spend_range_df["Spending Ranges (Per Student)"] = pd.cut(schools_by_spend_range_df["Per Student Budget"], bins,labels=spending_ranges)


schools_spend_range = schools_by_spend_range_df.groupby(["Spending Ranges (Per Student)"])
total_students_spend = schools_spend_range["Student ID"].count()
avg_math_score_spend = schools_spend_range["math_score"].mean()
avg_read_score_spend = schools_spend_range["reading_score"].mean()
number_passing_math_spend = schools_spend_range["Passing Math"].sum()
percent_passing_math_spend = schools_spend_range["Passing Math"].sum() / total_students_spend * 100
percent_passing_read_spend = schools_spend_range["Passing Read"].sum() / total_students_spend * 100
percent_passing_overall_spend = schools_spend_range["Passing Overall"].sum() / total_students_spend * 100

#Create new table with spending ranges
spend_range_sum_df = pd.DataFrame({"Average Math Score": avg_math_score_spend, "Average Reading Score": avg_read_score_spend, \
                                 "% Passing Math": percent_passing_math_spend,"% Passing Reading": percent_passing_read_spend, \
                                  "% Overall Passing": percent_passing_overall_spend})

spend_range_sum_df["Average Math Score"] = spend_range_sum_df["Average Math Score"].map("{:.2f}".format)
spend_range_sum_df["Average Reading Score"] = spend_range_sum_df["Average Reading Score"].map("{:.2f}".format) 
spend_range_sum_df["% Passing Math"] = spend_range_sum_df["% Passing Math"].map("{:.3f}%".format) 
spend_range_sum_df["% Passing Reading"] = spend_range_sum_df["% Passing Reading"].map("{:.3f}%".format)
spend_range_sum_df["% Overall Passing"] = spend_range_sum_df["% Overall Passing"].map("{:.3f}%".format) 

## Scores by School Spending

In [34]:
spend_range_sum_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<585,83.36,83.96,93.703%,96.687%,90.641%
$586-630,79.98,82.31,79.110%,88.513%,70.939%
$631-645,77.82,81.30,70.624%,82.600%,58.841%
$646+,77.05,81.01,66.231%,81.109%,53.529%


In [35]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
bins = [0,1000,2000,5000]
size_ranges = ["Small(<1k","Medium(1-2K)","Large(2-5K)"]

school_data_complete["School Size"] = pd.cut(schools_by_spend_range_df["size"], bins,labels=size_ranges)


schools_size_range = school_data_complete.groupby(["School Size"])
total_students_size = schools_size_range["Student ID"].count()
avg_math_score_size = schools_size_range["math_score"].mean()
avg_read_score_size = schools_size_range["reading_score"].mean()
number_passing_math_size = schools_size_range["Passing Math"].sum()
percent_passing_math_size = schools_size_range["Passing Math"].sum() / total_students_size * 100
percent_passing_read_size = schools_size_range["Passing Read"].sum() / total_students_size * 100
percent_passing_overall_size = schools_size_range["Passing Overall"].sum() / total_students_size * 100

#Create new table with spending ranges
size_range_sum_df = pd.DataFrame({"Average Math Score": avg_math_score_size, "Average Reading Score": avg_read_score_size, \
                                 "% Passing Math": percent_passing_math_size,"% Passing Reading": percent_passing_read_size, \
                                  "% Overall Passing": percent_passing_overall_size})

size_range_sum_df["Average Math Score"] = size_range_sum_df["Average Math Score"].map("{:.2f}".format)
size_range_sum_df["Average Reading Score"] = size_range_sum_df["Average Reading Score"].map("{:.2f}".format) 
size_range_sum_df["% Passing Math"] = size_range_sum_df["% Passing Math"].map("{:.3f}%".format) 
size_range_sum_df["% Passing Reading"] = size_range_sum_df["% Passing Reading"].map("{:.3f}%".format)
size_range_sum_df["% Overall Passing"] = size_range_sum_df["% Overall Passing"].map("{:.3f}%".format) 

## Scores by School Size

In [36]:
size_range_sum_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1k,83.83,83.97,93.952%,96.040%,90.137%
Medium(1-2K),83.37,83.87,93.617%,96.773%,90.624%
Large(2-5K),77.48,81.20,68.652%,82.125%,56.574%


In [37]:
#Perform the same operations as above, based on school type
schools_type_range = school_data_complete.groupby(["type"])
total_students_type = schools_type_range["Student ID"].count()
avg_math_score_type = schools_type_range["math_score"].mean()
avg_read_score_type = schools_type_range["reading_score"].mean()
number_passing_math_type = schools_type_range["Passing Math"].sum()
percent_passing_math_type = schools_type_range["Passing Math"].sum() / total_students_type * 100
percent_passing_read_type = schools_type_range["Passing Read"].sum() / total_students_type * 100
percent_passing_overall_type = schools_type_range["Passing Overall"].sum() / total_students_type * 100

#Create new table with spending ranges
type_range_sum_df = pd.DataFrame({"Average Math Score": avg_math_score_type, "Average Reading Score": avg_read_score_type, \
                                 "% Passing Math": percent_passing_math_type,"% Passing Reading": percent_passing_read_type, \
                                  "% Overall Passing": percent_passing_overall_type})

type_range_sum_df["Average Math Score"] = type_range_sum_df["Average Math Score"].map("{:.2f}".format)
type_range_sum_df["Average Reading Score"] = type_range_sum_df["Average Reading Score"].map("{:.2f}".format) 
type_range_sum_df["% Passing Math"] = type_range_sum_df["% Passing Math"].map("{:.3f}%".format) 
type_range_sum_df["% Passing Reading"] = type_range_sum_df["% Passing Reading"].map("{:.3f}%".format)
type_range_sum_df["% Overall Passing"] = type_range_sum_df["% Overall Passing"].map("{:.3f}%".format) 

## Scores by School Type

In [24]:
type_range_sum_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.41,83.90,93.702%,96.646%,90.561%
District,76.99,80.96,66.518%,80.905%,53.696%
